In [1]:
!pip install pypdf
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -U langchain-community
!pip install -U langchain transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
import re
import json
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# Function to clean folder and file names
def clean_name(name):
    name = re.sub(r'\s*', '', name)
    name = re.sub(r'\.pdf$', '', name, flags=re.IGNORECASE)
    return name.strip()

# Function to get source link based on file name
def get_source_link(file_name):
    # Extract the key part
    for key in url_mapping:
        if key in file_name:  # Match partial key
            return url_mapping[key]
    return backup_source_link(folder_name)

BASE_URL = "https://www.bostonpublicschools.org/Page/5357"
def backup_source_link(folder_name):
    abbreviation = folder_name.split('(')[-1].replace(')', '')
    return f"{BASE_URL}#{abbreviation}"

In [20]:
# Path to the main dataset directory
dataset_path = '/content/drive/MyDrive/dataset'
new_dataset_path = '/content/drive/MyDrive/new_dataset'

# Initialize list to store all chunks and policies hashmap
all_chunks = []
policies = {}
folder_count = 0
pdf_count = 0

# Function to process PDF
def process_pdf(pdf_path, folder_name):
    global pdf_count
    try:
        # Loading pdf
        loader = PyPDFLoader(file_path=pdf_path)
        docs_before_split = loader.load()

        if len(docs_before_split) == 0:
            print(f"Warning: No content found in {pdf_path}")
            return

        # Initialize the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
        )

        # Split the documents into chunks
        docs_after_split = text_splitter.split_documents(docs_before_split)

        if len(docs_after_split) == 0:
            print(f"Warning: No chunks created from {pdf_path}")
            return

        # Clean folder and file names
        clean_folder_name = clean_name(folder_name)
        clean_file_name = clean_name(os.path.basename(pdf_path))

        # Add to policies hashmap
        abbreviation = folder_name.split('(')[-1].replace(')', '')
        if abbreviation not in policies:
            policies[abbreviation] = []
        policies[abbreviation].append(clean_file_name)

        # Prepare chunks with metadata
        for i, doc in enumerate(docs_after_split):
            chunk_data = {
                'folder_name': clean_folder_name,
                'file_name': clean_file_name,
                'chunk_id': i + 1,
                'content': doc.page_content,
                'source_link': get_source_link(clean_file_name),
                'backup_link': backup_source_link(clean_folder_name)
            }
            all_chunks.append(chunk_data)

        pdf_count += 1
        print(f"Processed {pdf_path}, created {len(docs_after_split)} chunks.")
    except Exception as e:
        print(f"Error processing {pdf_path}: {str(e)}")

# Walk through the dataset directory and process all PDF files
for root, dirs, files in os.walk(new_dataset_path):
    if files:
        folder_count += 1
    for file_name in files:
        if file_name.endswith('.pdf'):
            pdf_path = os.path.join(root, file_name)
            folder_name = os.path.basename(root)
            process_pdf(pdf_path, folder_name)

# Save chunks to a JSON file
if all_chunks:
    output_path = '/content/chunked_data_all_folders_with_links.json'
    with open(output_path, 'w') as json_file:
        json.dump(all_chunks, json_file, indent=4)
    print(f"All PDF chunks with links have been saved to {output_path}")
else:
    print("No chunks were created. Please check the input files.")

# Save policies hashmap to a JSON file
policies_output_path = '/content/policies_hashmap.json'
with open(policies_output_path, 'w') as json_file:
    json.dump(policies, json_file, indent=4)
print(f"Policies hashmap has been saved to {policies_output_path}")

# Print statistics
print(f"Number of folders traversed: {folder_count}")
print(f"Number of PDFs processed: {pdf_count}")

Processed /content/drive/MyDrive/new_dataset/CAO-06  GPA Calculation Method.pdf, created 8 chunks.
All PDF chunks with links have been saved to /content/chunked_data_all_folders_with_links.json
Policies hashmap has been saved to /content/policies_hashmap.json
Number of folders traversed: 1
Number of PDFs processed: 1


In [21]:
# Path to the JSON file containing chunked data
json_file_path = '/content/chunked_data_all_folders_with_links.json'

# Load the JSON data
with open(json_file_path, 'r') as json_file:
    chunked_data = json.load(json_file)

# Initialize list to store documents
documents = []

# Process each entry in the JSON data
for entry in chunked_data:
    # Extract fields from JSON entry
    original_content = entry['content']
    folder_name = entry['folder_name']
    file_name = entry['file_name']
    source_link = entry.get('source_link', 'Unknown Source Link')
    backup_link = entry.get('backup_link', 'Unknown BackUp Link')

    # Create Document objects for each entry with metadata
    doc = Document(
        page_content=original_content,
        metadata={
            'folder_name': folder_name,
            'file_name': file_name,
            'source_link': source_link,
            'backup_link': backup_link
        }
    )
    documents.append(doc)

print(len(documents))

8


In [11]:
import faiss
import json
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [12]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

<ipython-input-12-a34a09b47e7d>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
db = FAISS.from_documents(documents, embeddings)

In [17]:
# Paths for FAISS index and metadata files
index_file = "/content/drive/MyDrive/faiss/faiss_index"
meta_file = "/content/drive/MyDrive/faiss/faiss_metadata.json"

# Function to save FAISS index and metadata
def save_faiss_index_and_metadata(faiss_db):
    # Save FAISS index
    faiss.write_index(faiss_db.index, index_file)
    print(f"FAISS index saved to {index_file}")

    # Serialize Document objects to JSON-serializable format
    metadata_dict = {
        key: {
            "page_content": value.page_content,
            "metadata": value.metadata
        }
        for key, value in faiss_db.docstore._dict.items()
    }

    with open(meta_file, 'w') as f:
        json.dump(metadata_dict, f, indent=4)
    print(f"Metadata saved to {meta_file}")

save_faiss_index_and_metadata(db)

FAISS index saved to /content/drive/MyDrive/faiss/faiss_index
Metadata saved to /content/drive/MyDrive/faiss/faiss_metadata.json


In [18]:
def load_faiss_from_files(index_path, metadata_path, embedding_function):
    try:
        # Load FAISS index
        loaded_index = faiss.read_index(index_path)
        print(f"FAISS index loaded from {index_path}")

        # Load metadata
        with open(metadata_path, "r") as f:
            metadata_dict = json.load(f)
        print(f"Metadata loaded from {metadata_path}")

        # Ensure consistency between the index and metadata
        if loaded_index.ntotal != len(metadata_dict):
            raise ValueError(
                f"Mismatch between FAISS index vectors ({loaded_index.ntotal}) and metadata entries ({len(metadata_dict)})."
            )

        # Reconstruct the document store
        docstore = InMemoryDocstore({
            key: Document(page_content=value["page_content"], metadata=value["metadata"])
            for key, value in metadata_dict.items()
        })

        # Recreate the index_to_docstore_id mapping
        index_to_docstore_id = {i: key for i, key in enumerate(metadata_dict.keys())}

        # Recreate the FAISS database
        faiss_db = FAISS(
            index=loaded_index,
            docstore=docstore,
            index_to_docstore_id=index_to_docstore_id,
            embedding_function=embedding_function
        )
        print("FAISS database successfully reconstructed.")
        return faiss_db
    except Exception as e:
        print(f"Error loading FAISS data: {e}")
        return None

file_index = "/content/drive/MyDrive/faiss/faiss_index"
meta_file = "/content/drive/MyDrive/faiss/faiss_metadata.json"

db = load_faiss_from_files(file_index, meta_file, embeddings)

FAISS index loaded from /content/drive/MyDrive/faiss/faiss_index
Metadata loaded from /content/drive/MyDrive/faiss/faiss_metadata.json
FAISS database successfully reconstructed.


In [19]:
# Perform a similarity search with the question
question = "What are the policies for enrollment?"
searchDocs = db.similarity_search(question)

# Loop through relevant documents and print their content, links, source links, and file names
for i, doc in enumerate(searchDocs):
    print(f"Document {i+1} Content:\n{doc.page_content}\n{'-'*100}")

    # Retrieve and print file name
    if 'file_name' in doc.metadata:
        print(f"File Name: {doc.metadata['file_name']}")
    else:
        print(f"File Name: None")

    # Retrieve and print source link
    if 'source_link' in doc.metadata:
        print(f"Source Link: {doc.metadata['source_link']}")
    else:
        print(f"Source Link: None")

    # Retrieve and print backup link
    if 'backup_link' in doc.metadata:
        print(f"Backup Link: {doc.metadata['backup_link']}")
    else:
        print(f"Backup Link: None")

    print("="*100)

Document 1 Content:
repeated annually or when a student transfers classrooms or 
schools.  
• Ensure that there is a contingency plan in place for all 
school-related venues where substitutes are utilized. 
• Encourage the students to eat all meals and snacks fully and 
on time. Be flexible with time requirements for eating and 
provide the parent or guardian with the carbohydrate 
menu. 
• Make certain that emergency communication devices (e.g., 
walkie-talkie, intercom, cell phone, etc.) are always present 
and functional. 
• Participate in the teams that develop and implement the 
student’s Section 504 Plan, other education plan, or 
individualized education program. Contribute to IEP, and 
504 implementation of diabetes related issues, where
----------------------------------------------------------------------------------------------------
File Name: SHS-21DiabetesPolicy
Source Link: https://drive.google.com/file/d/13x80o_7kQavqdXlbvNl7E9o42W9tfecR/view
Backup Link: https://www.bo

In [23]:
import numpy as np

def ensure_index_idmap(faiss_index):
    # If the index is already an IndexIDMap, return it as-is
    if isinstance(faiss_index, faiss.IndexIDMap):
        return faiss_index

    # If the index is empty, wrap it in IndexIDMap
    if faiss_index.ntotal == 0:
        return faiss.IndexIDMap(faiss_index)

    # Recreate an empty IndexIDMap with the same structure
    dimension = faiss_index.d
    new_index = faiss.IndexIDMap(faiss.IndexFlatL2(dimension))

    # Transfer the existing data
    xb = faiss_index.reconstruct_n(0, faiss_index.ntotal)
    ids = np.arange(faiss_index.ntotal, dtype='int64')
    new_index.add_with_ids(xb, ids)

    print(f"Transferred {faiss_index.ntotal} vectors to a new IndexIDMap.")
    return new_index

def add_to_faiss(faiss_db, embeddings, documents):
    import numpy as np

    # Ensure the FAISS index is wrapped in IndexIDMap
    faiss_db.index = ensure_index_idmap(faiss_db.index)

    # Add documents to the FAISS index and docstore
    for i, doc in enumerate(documents):
        try:
            # Generate a unique vector ID for the document
            vector_id = hash(doc.metadata["file_name"] + str(i)) % (2**63 - 1)

            # Generate embedding for the document content
            vector = embeddings.embed_query(doc.page_content)

            # Add embedding to the FAISS index
            faiss_db.index.add_with_ids(
                np.array([vector]).astype("float32"),
                np.array([vector_id], dtype="int64")
            )

            # Add document metadata to the docstore
            faiss_db.docstore._dict[str(vector_id)] = doc

            # Update the index_to_docstore_id mapping
            faiss_db.index_to_docstore_id[vector_id] = str(vector_id)

        except Exception as e:
            print(f"Error adding document {doc.metadata.get('file_name', 'unknown')} to FAISS: {e}")

    print(f"Added {len(documents)} documents to the FAISS index.")
    return faiss_db

new_pdf_path = "/content/CAO-06  GPA Calculation Method.pdf"
folder_name = "Academics(CAO)"
db = add_to_faiss(db, embeddings, documents)

Transferred 3747 vectors to a new IndexIDMap.
Added 8 documents to the FAISS index.


In [25]:
def remove_from_faiss(faiss_db, file_name):
    # Identify all keys (IDs) associated with the file_name
    keys_to_remove = []
    for key, doc in faiss_db.docstore._dict.items():
        if doc.metadata.get("file_name") == file_name:
            keys_to_remove.append(key)

    # If no matching content is found, return a message
    if not keys_to_remove:
        print(f"No content found in FAISS index for file_name: {file_name}")
        return faiss_db

    # Remove from FAISS index and docstore
    for key in keys_to_remove:
        del faiss_db.docstore._dict[key]
    faiss_db.index.remove_ids(np.array(list(map(int, keys_to_remove)), dtype='int64'))

    print(f"Removed {len(keys_to_remove)} chunks related to file_name: {file_name}")
    return faiss_db

file_name_to_remove = clean_name("CAO-06  GPA Calculation Method.pdf")
db = remove_from_faiss(db, file_name_to_remove)

Removed 8 chunks related to file_name: CAO-06GPACalculationMethod


In [26]:
# Perform a similarity search with the question
question = "What are some methods to calculate GPA?"
searchDocs = db.similarity_search(question)

# Loop through relevant documents and print their content, links, source links, and file names
for i, doc in enumerate(searchDocs):
    print(f"Document {i+1} Content:\n{doc.page_content}\n{'-'*100}")

    # Retrieve and print file name
    if 'file_name' in doc.metadata:
        print(f"File Name: {doc.metadata['file_name']}")
    else:
        print(f"File Name: None")

    # Retrieve and print source link
    if 'source_link' in doc.metadata:
        print(f"Source Link: {doc.metadata['source_link']}")
    else:
        print(f"Source Link: None")

    # Retrieve and print backup link
    if 'backup_link' in doc.metadata:
        print(f"Backup Link: {doc.metadata['backup_link']}")
    else:
        print(f"Backup Link: None")

    print("="*100)

Document 1 Content:
is treated like an A.” To address this requirement, the 12-point 
grade scale will be rescaled from 0-12 to 0-11, where 0 points 
represent an F and 11 points represent both A+ and A. This will 
result in the following crosswalk from letter marks to point values 
used in the GPA calculation. 
 
Letter 
Mark 
A+ A A- B+ B B- C+ C C- D+ D D- F 
Point 
Value 
11 11 10 9 8 7 6 5 4 3 2 1 0 
 
Students with grade 5 transcripts from Boston Public Schools 
receive marks on multiple standards for each subject area using a 
1-4 grading scale.  The marks in Reading, Writing, and Math will be 
converted to a 12-point scale for the purpose of calculating an 
“overall” mark in the subject area (ELA and Math).  If the “overall” 
mark in either subject is above 11, the number will be rounded 
down to 11 to align with the 1–11 point scale.
----------------------------------------------------------------------------------------------------
File Name: AMT-01ExamSchoolApplicationandAdm

In [4]:
# Define the URL mapping
url_mapping = {
    "ATH-01": "https://drive.google.com/file/d/1aH_x4Q7_yfSGjqbMBpPP5-M3yII4mc4o/view",
    "ATH-02": "https://drive.google.com/file/d/1VRfs42DpguRuaCKQEyGxv9Xs_hFbUpcO/view",
    "COM-01": "https://drive.google.com/file/d/1JrJBRQ87JvYvlLe0w5d-BQ0K-2eq3wjM/view",
    "COM-02": "https://drive.google.com/file/d/1Nun6F0fivaj38RKxzURNca_ptsEjNlua/view",
    "ACA-18": "https://drive.google.com/drive/folders/1XNOOmnWE4VMQ-I1stRUkAhogGw9t0I5G",
    "AMT-01": "https://drive.google.com/file/d/1sMHkNfYGn7r8VGEVCToGula73oEtYgwN/view",
    "AMT-03": "https://drive.google.com/file/d/1hEpcnEnD17bfEOGhuAgeY1G-aMUctJib/view",
    "AMT-04": "https://drive.google.com/file/d/1XJrumxGVBFNxG__pHfTKPYI6uQwuf8G5/view",
    "AMT-05": "https://drive.google.com/file/d/1xB0sAawkLC1HAhM6rbqFzh31CF_5ro03/view",
    "AMT-06": "https://drive.google.com/file/d/1fgaUp4Pn374_lBuJaW1mBZG_JUnWUGy_/view",
    "AMT-07": "https://drive.google.com/file/d/158Utqa8XW8gth30_BexWoRielUKJuke9/view",
    "CAO-01": "https://drive.google.com/drive/folders/1huzLTVFKFjveOeqp0_-AcTf-jYphCld3",
    "CAO-03": "https://drive.google.com/drive/folders/1-NCNpSa8UwYw_0MQdhmM-VV5lcBKtGEU",
    "CAO-05": "https://drive.google.com/drive/folders/1HVAuaNXNlnowmZKjhXsGRHPcaW6sf3-g",
    "CAO-06": "https://drive.google.com/drive/folders/1ThM6wTF5LH2fCiMWDrSEiJKXutrc2u5x",
    "CAO-07": "https://drive.google.com/drive/folders/1kxczLy6UXIbujEjqm6GOZtfV9laCVV0K",
    "CAO-08": "https://drive.google.com/drive/folders/1HpUfPePRLPCVkn_OP8FO9U4UtVah-U2g",
    "CAO-22": "https://drive.google.com/drive/folders/1Dbicrju9SkB4GQ8ou9_8NS9Ke240k4p2",
    "CAO-23": "https://drive.google.com/drive/folders/1e-qZveyEspiHKe3WnHlkF-_SXUMml8Gq",
    "CAO-24": "https://drive.google.com/drive/folders/1GOj3aqWOhe0WJoZU9X2mEOSEYWniowoG",
    "CAO-25": "https://drive.google.com/drive/folders/1bgSWMoxrTIqer_Uj6dArbc_yB5r07AqF",
    "CAO-27": "https://drive.google.com/drive/folders/1DdQVFUzs8FgfXjjJGBWhvmSIvEPWYVqv",
    "EL-04": "https://drive.google.com/file/d/1YWFdLk26LntE-MjUuW6pLjsg7qSDtNc8/view",
    "EL-06": "https://drive.google.com/file/d/1thfVB-Lhr6AC8l3gbx9wR_LJ-3bk7H-L/view",
    "EL-07": "https://drive.google.com/file/d/1vVZBe5fMfdRhlQeP2cOiUYoVlSFZ0Tu-/view",
    "EQT-01": "https://drive.google.com/drive/folders/1Z8sExAkw1ZYayk9OX4InngZHn7UQ-1GX",
    "EQT-02": "https://drive.google.com/drive/folders/1xqHNd5yN_rx_mStyh9Jz7Xz4keHGib44",
    "EQT-03": "https://drive.google.com/drive/folders/1Pe2hh44rr5bsveLmf0Z-_rEzeTZkl-OY",
    "EQT-04": "https://drive.google.com/drive/folders/12efP8zQjPFsMMFZAN3jgYZMUX4p2kHyh",
    "EQT-05": "https://drive.google.com/drive/folders/1KKRzKOiOSHpaPrcMOg-dI0hWfjDxz03j",
    "EQT-06": "https://drive.google.com/drive/folders/16Zj_B-UDQd0da_XNWyfnva6kkx-aBKQS",
    "EQT-07": "https://drive.google.com/drive/folders/1ITmCZNJ3OkfKmxygiCCWl1zZ3MrqzAPd",
    "EQT-08": "https://drive.google.com/drive/folders/1IC4yTlHZsLBxq-6YokiNPhJsN5ntC6ms",
    "EQT-09": "https://drive.google.com/drive/folders/1TcqkqmQmLIIFiNoz_syOmjSh-YAHkxQl",
    "EQT-10": "https://drive.google.com/drive/folders/12lvt_AyOhYHVuki08N_3kHDUVFzhp5HO",
    "FAM-01": "https://drive.google.com/file/d/1Ppltgzjz9uUMM2uW3q9dUQJDZ65D_KO4/view",
    "FAM-02": "https://drive.google.com/file/d/158OAoHtvQdrniiu40HImPIiEtDCC7rJw/view",
    "FAM-03": "https://drive.google.com/file/d/1DUa6lzjoi_65P_Iceq7zyFjOWcuEPloz/view",
    "FAM-04": "https://drive.google.com/file/d/15RonpFA4W7MI57QB3_7YxM8GMrHIc1k5/view",
    "FAM-05": "https://drive.google.com/file/d/1Yxfz3sQ9Ob-FX1MwbjHJIOR2bm4zUkaL/view",
    "FAM-06": "https://drive.google.com/file/d/1XBoUbxJns6RnOwLbVzFOvDwYrzdz2x3L/view",
    "FAM-07": "https://drive.google.com/file/d/181grRc0Qm1wunp-SbK4yZSisdWUg-MfL/view",
    "FAM-08": "https://drive.google.com/file/d/1rTUwL0SY3tBMPNoIfi5UkVSQRzbu9gyT/view",
    "FIN-01": "https://drive.google.com/file/d/1pLrKq8DDDiIigEBpZuvzPZgbQi524Irx/view",
    "FIN-02a": "https://drive.google.com/file/d/1sKflA_u87K3kYXuwE5hZt_3mXBLRyCAU/view",
    "FIN-02b": "https://drive.google.com/file/d/1aV_RsESDq2PtfaciK_CvVwQNjqn0uVv-/view",
    "FIN-03": "https://drive.google.com/file/d/1AxdMc3cgaoqR6aAhCPzzI9f_lJYmX_VI/view",
    "FIN-04": "https://drive.google.com/file/d/1jQ50oYTgMJEIcnFN_eH8AKxy7MjBFaOC/view",
    "FIN-07": "https://drive.google.com/file/d/1fOGZFif8xWwqMjBKuSxjLisD8cUMtEwh/view",
    "FIN-09": "https://drive.google.com/file/d/17_--vm5j21c3fgnkn9ZuiaZ_80_A--Lf/view",
    "FIN-10": "https://drive.google.com/file/d/1zt5hrWNvO6jiTf2K54Pmm8-j_Jntztqk/view",
    "FIN-11": "https://drive.google.com/file/d/1XJJiUpQRNOzILU-BNey-zkCvFMmllkxb/view",
    "FIN-12": "https://drive.google.com/file/d/1vL67Ryo9P6rtWbbfsBOVDpQpesht9kzZ/view",
    "FIN-14": "https://drive.google.com/file/d/1Dmgw_5-W8ifS6WUBn4SyCYJCMoF5ZM-W/view",
    "FIN-16": "https://drive.google.com/file/d/1E70jLqE_7fJjFcAjmxK5S6oX_NVfHOv4/view",
    "FIN-19": "https://drive.google.com/file/d/17GRe2UPMjttYAJgSp8n-VM4oD6-NJbq0/view",
    "FIN-20": "https://drive.google.com/file/d/1lEzhFxMwkSxgpgQuy6ZUPDT_NHK74gev/view",
    "FIN-21": "https://drive.google.com/file/d/1C77WJzv6bA_DAqmk6xlurf-9PcdArTYH/view",
    "FMT-01": "https://drive.google.com/file/d/1bQlDaxIKlS9coPOLTXsRN1N3OlzWSoCJ/view",
    "FMT-02": "https://drive.google.com/file/d/1Zwno4HydEGdSzfvkDJ7uEx4OeooMPcMF/view",
    "FMT-03": "https://drive.google.com/file/d/1738orkSnXIVEpr56NkfqQ7tMn1sOhJ4K/view",
    "FMT-04": "https://drive.google.com/file/d/14TfH360t0phHPHzTfuUugzUoDSR_DqjD/view",
    "FMT-05": "https://drive.google.com/file/d/13iXUfcrAXeQP0in09cA-YHmtf7SuRyT6/view",
    "FMT-08": "https://drive.google.com/file/d/1PyyfwMR5xMFb5rvj0TWd1ARjhZNY_biV/view",
    "FMT-09": "https://drive.google.com/file/d/1yW9E0PBVETYyd3Xu1Rf94AWhLBAI2efQ/view",
    "FMT-10": "https://drive.google.com/file/d/1lButB53moRjvYyQfuVnyG0jGBOC296sq/view",
    "FMT-11": "https://drive.google.com/file/d/1PdZzSR-hFKXfrdAGpdOwqf31lXoJwlRY/view",
    "FMT-12": "https://drive.google.com/file/d/1PCoV3DzKnwt0Of3PtZ3LrrDPrACRdxtD/view",
    "FMT-15": "https://drive.google.com/file/d/1mPINlDjRMn17VzIBbhWk7tQa464s2bzl/view",
    "FMT-18": "https://drive.google.com/file/d/1_jAqzwZzoMtS-KQEExhT5nonFda7gqbx/view",
    "FMT-19": "https://drive.google.com/file/d/1gIN6N7i6OdPIPoWRLw6t2vJbHjXtVpVy/view",
    "FMT-20": "https://drive.google.com/file/d/16ikP51nwJO58_BikMlZHxN152J2g0EUd/view",
    "FNS-02": "https://drive.google.com/file/d/1VxLTLmK3rTa8BRrGablCSRJ_ClEDGxVI/view",
    "FNS-03": "https://drive.google.com/file/d/1WJeEtMsdzZbVx-pIHvpIuou-uyo6fbvR/view",
    "FNS-04": "https://drive.google.com/file/d/1iQFifLWXwefooAOlyKZTCBbBDIvyuWR1/view",
    "FNS-06": "https://drive.google.com/file/d/1FA2GijhLgdjxslmzQQ93rszJU0XkW69A/view",
    "FSE-01": "https://drive.google.com/drive/folders/10wXriKyfMKg8nt3FGWxnkaMFR8AEC2LF",
    "FSE-02": "https://drive.google.com/file/d/1vN6bIecqoIk46f_qybgb9HHyfXL5O0se/view",
    "FSE-03": "https://drive.google.com/file/d/1VFPSHncUdDWBkhHpBByL-PNUlAq-MpaF/view",
    "FSE-04": "https://drive.google.com/file/d/15D7bBLLtV5AR9_0xoP6UztcH-bDX7_6X/view",
    "FSE-05": "https://drive.google.com/file/d/1JMTHeOLx4S8mdC3PJPgZtAyqBCTQusrK/view",
    "FSE-06": "https://drive.google.com/file/d/1HC4uTS7zRT4g-cxmAdHsKNP9UUpNZi3G/view",
    "FSE-07": "https://drive.google.com/file/d/1-b_CqanFp-HVdIy-Gf-5vR9uF2LCgOk9/view",
    "FSE-08": "https://drive.google.com/file/d/1cQOOmQRzRutNAbbEoTGTrsACSvezTvLD/view",
    "HRS-HS02": "https://drive.google.com/file/d/10B4DGDW4x0rD42IyIfgugWMI9qYPkpcI/view",
    "HRS-HS04": "https://drive.google.com/file/d/1UkFqwECZapwrtRXIHzWIWmzq7sac6xIY/view",
    "HRS-HS06": "https://drive.google.com/file/d/1XdL_rU44reTNpXTb1ICytlZQDrPVF65_/view",
    "HRS-HS07.01": "https://drive.google.com/file/d/1h-zYiF62_9uNGVuEAweqpKa-LlfPY-K4/view",
    "HRS-L01": "https://drive.google.com/file/d/1IZQ9ElFzcNfsyECNo6iPwGmIRX8ihsGa/view",
    "HRS-L02": "https://drive.google.com/file/d/1Fvy2gkuA0TJC0MJdpU-KJ7eN-x4b_33l/view",
    "HRS-L03": "https://drive.google.com/file/d/16B9RTIXiKiHRn2a3LyqS_mfpi6GWQv_n/view",
    "HRS-PM01": "https://drive.google.com/file/d/1TQtYp9_oSw1DMHEYQsHNBOAYJUzo3pKP/view",
    "HRS-PM02": "https://drive.google.com/file/d/1yTEQTwZDA3rEADhDNztao8bupHlYB123/view",
    "HRS-PM02A": "https://drive.google.com/file/d/1KV1rqDyFPV-LCY81e3ntl_r3DRnClYn8/view",
    "HRS-PM03": "https://drive.google.com/file/d/1YcjBA_TVV08gerlmVDb1Rd6wtS32VJ4e/view",
    "HRS-PM04": "https://drive.google.com/file/d/1u-bCHoqUHaYNYgw01chsQostqNDktb2-/view",
    "HRS-PM05": "https://drive.google.com/file/d/1ZvQc4w018itaB64FpJFbK7bHNIY61rof/view",
    "HRS-PM06": "https://drive.google.com/file/d/1qjy_nf9F-0zF4rRT7rdjL2fB67cb1HUT/view",
    "HRS-PM07": "https://drive.google.com/file/d/1dsAXq5lwJ4th4rwaJK_6x3zhNDpORaxC/view",
    "HRS-PM07A": "https://drive.google.com/file/d/1r3niNCtd975l7U7sSQYnkECm15jPAfjJ/view",
    "HRS-PM08": "https://drive.google.com/file/d/10_W1hXOwt7c74wdXYcjf2e82LwBv9fBY/view",
    "HRS-PM09": "https://drive.google.com/file/d/1P9qWEGNG6iQoF3TKERyYQHeUKFLsNSS9/view",
    "HRS-PM10": "https://drive.google.com/file/d/1bKri-l2mMmpRfic6-MZ57Nd_GfUPpgN-/view",
    "HRS-PP01": "https://drive.google.com/file/d/1vrprFGaGOXUGpNv8P4dYvqSSgNHsf2J7/view",
    "HRS-PP05": "https://drive.google.com/file/d/1ePsaAXm5WOgLt3KYUpuVQDI4JfQj6GjH/view",
    "HRS-PP06": "https://drive.google.com/file/d/1xF89akowuh05oZolNkzbAKhbz4zL73rS/view",
    "HRS-PP07": "https://drive.google.com/file/d/1gluiHzgMz8fw5K8zs5zro6nxoxl7U--C/view",
    "HRS-PP08": "https://drive.google.com/file/d/1gE8oiBSb3RuZN7wBOmo0fP0fLvZGSHpS/view",
    "HRS-PP09": "https://drive.google.com/file/d/1Z4awZwQZPC5bs4-nzAM-MU-VtrwdtsdO/view",
    "HRS-PP11": "https://drive.google.com/file/d/1lqa7o8BTT1u6EwoL2GQF9h0VKlFWtniK/view",
    "HRS-PP12": "https://drive.google.com/file/d/10DqlDTejhB1LP4GMX_hkKJxgIY0xuQs1/view",
    "HRS-PP13": "https://drive.google.com/file/d/1WfgCjt7CYvXJw-uJKdNiOIenk00t7rpY/view",
    "HRS-PP13A": "https://drive.google.com/file/d/1ItWEjelmAub33iBM4Z7TASKumXhn2Q1q/view",
    "HRS-PP14": "https://drive.google.com/file/d/15T5B-gktHZLJ4SbpWIABE8F8w5PKy9F4/view",
    "HRS-PP15": "https://drive.google.com/file/d/1WDNCc8Kes8THYqJrL1gKfz-MThr_iaOg/view",
    "HRS-PP16": "https://drive.google.com/file/d/1HvyLZIGBqP7jwctoPMhuiB_nCgnoEA5r/view",
    "HRS-PP17": "https://drive.google.com/file/d/1bi0MxV8HuxuaxwlCZYGUCafxqJG9gfzU/view",
    "HRS-PP19": "https://drive.google.com/file/d/1v8oKbayV652Lf6JjPAo-6NdCY3kzub-f/view",
    "HRS-PP20": "https://drive.google.com/file/d/1yIA_6ixUsB9stTDMGqPKX0SqvulGW_up/view",
    "HWD-01": "https://drive.google.com/file/d/1rsrvtkOdvgvcc4CMZgy2KBsNaocwkzjb/view",
    "HWD-02": "https://drive.google.com/file/d/1etGOTQZuYGtcISAuzVPBpho3Li_9M0ZJ/view",
    "HWD-03": "https://drive.google.com/file/d/1BZN4J_1BvMheW-E9orazPHIiLKXN9zxx/view",
    "HWD-04": "https://drive.google.com/file/d/1Vzt84GB9aIGYO2toPFBqhoWI0Vxr9dAZ/view",
    "HWD-06": "https://drive.google.com/file/d/1bxrk-vqbESEH-I-rxdw6q8yiwGvHoWN8/view",
    "LGL-01": "https://drive.google.com/file/d/1ynMmViUIVaCzlNlfUU7_5OTUtyG5iIgM/view",
    "LGL-03": "https://drive.google.com/file/d/1_2TybcRFKvz34WDkXY0p-G-sjcTisP5D/view",
    "LGL-04": "https://drive.google.com/drive/folders/15Ga6xXJ98ifGgp8sk5I_HsWPQPnzfFSq",
    "LGL-05": "https://drive.google.com/file/d/1lPWU62wlDzYb9MwaZ4Chfs6CoodJ80dr/view",
    "LGL-06": "https://drive.google.com/file/d/13seN0T59ZA_IsLqXWLqnPJywGMuShMnb/view",
    "LGL-07": "https://drive.google.com/drive/folders/1o0lasKvkXmK1Gy9Y1Kq8E5hWyNWsE94Y",
    "LGL-08": "https://drive.google.com/file/d/1alool4krGzAgp3dnJZQAzJEorZbaTIbP/view",
    "LGL-09": "https://drive.google.com/file/d/11a6SDKrsi0PpIgI539k0ieG9F98G0po-/view",
    "LGL-10": "https://drive.google.com/file/d/1C_71dnBF5SnGLcVLNCFWjBl4MKkjhLrB/view",
    "LGL-14": "https://drive.google.com/file/d/1g9Xrs5hDEoCkvAn2d7S10zT-xCsf-Ipv/view",
    "LGL-15": "https://drive.google.com/file/d/190qznb-ZLibCLc5p7KIu1gbPBg3OyfP1/view",
    "LGL-16": "https://drive.google.com/file/d/1uoOWP8svQtwOGA8KqX0ZrQOiX5y7r7Ym/view",
    "LGL-17": "https://drive.google.com/file/d/183UnPyloCcYR6ugf0BekGD6oMqPdM_12/view",
    "LGL-18": "https://drive.google.com/file/d/1eIJlW2VFBFwh6h0Qt_dxtFdk18y-vPAt/view",
    "LGL-19": "https://drive.google.com/file/d/1fjXh28CDzUCl96L0HsPJZbDncCe1-w53/view",
    "LGL-20": "https://drive.google.com/drive/folders/1DfVdIZ4GF4oBmA_zNIOtO4649uijpbu8",
    "LGL-21": "https://drive.google.com/file/d/1mVZ_wtBruBY4ehVemVhLVtNRpslA5w8-/view",
    "LGL-22": "https://drive.google.com/file/d/1xDXO5s-c7h5sy_Jl0-I4gMw3l9S-EtR6/view",
    "ODA-01": "https://drive.google.com/file/d/11TsG3W91ZqVKBxTfmAiR72LOGpus0f_g/view",
    "ODA-02": "https://drive.google.com/file/d/14nhWVNCrEUQZRvjglwItvw2gMH3oaZbS/view",
    "ODA-03": "https://drive.google.com/file/d/14Ozk7pEiKHFrg_-3dgUYNPTtrDMZSRSq/view",
    "ODA-04": "https://drive.google.com/file/d/1w42nnVXO-sjpMPoK51i6AWbhbid5Vbnw/view",
    "ODA-05": "https://drive.google.com/file/d/1BT_Xv0yJfedkYHlGWSmWFSNtjpvyUsoi/view",
    "ODA-06": "https://drive.google.com/file/d/1Qa7H9xqJyE8twkkpuUBmM612HfHdE_VZ/view",
    "ODA-07": "https://drive.google.com/file/d/1MM69zNpH--fEI9hH4q4hYOEYPsMt3uSm/view",
    "OIIT-01": "https://drive.google.com/file/d/1c55h6BML_PPBCARZhbXjK_Clsw373G1_/view",
    "OIIT-02": "https://drive.google.com/file/d/1BQ1YFK1s9kzMTsdm-8hNc3r87ECBv0ZG/view",
    "OIIT-03": "https://drive.google.com/file/d/129Gf0dqObiJEm6ksGkuWIgkaMk5AIr-O/view",
    "SAF-01": "https://drive.google.com/file/d/1NrULbCUFhbzLts3Lm1d1pKNLzXvu_U0x/view",
    "SAF-02": "https://drive.google.com/file/d/13ZDBh8moRdR3SVg0uzgTNABLkc4qzuo0/view",
    "SAF-03": "https://drive.google.com/file/d/1pudS9x-G0sJY2dvohHacQs3Ydv2TMgX5/view",
    "SAF-04": "https://drive.google.com/drive/folders/1ohHzixZdvg7LtA5PDO7XMTz_xbdW-IHk",
    "SAF-07": "https://drive.google.com/file/d/1nKQTl_GDl9xYJV_GC6Z8-AqtEXWcz0vF/view",
    "SAF-08": "https://drive.google.com/drive/folders/1LRBTrHScoKA4SZThrCgLAxxERPSRd2by",
    "SAF-09": "https://drive.google.com/drive/folders/1OfUmu7Xh0SROMIxFIT4fd4ONjjGHAq1m",
    "SAF-12": "https://drive.google.com/file/d/1EMBtpt88uf6v3H4XFNI5fzKD9nHIEQ57/view",
    "SCP-01": "https://drive.google.com/file/d/1EKjqLfdWUJG9JDHd2Wp0ihrt0f59qpqh/view",
    "SHS-01": "https://drive.google.com/file/d/1us4agrVCDfO5M-vkTlGh5SpDJVgP-LNl/view",
    "SHS-04": "https://drive.google.com/file/d/1ynUdSIjTH-avvsE68pkkvCRTwaSWzp_V/view",
    "SHS-05": "https://drive.google.com/file/d/1KG40PmfOQc5-OtE_vsw89x1cdOcUCWJX/view",
    "SHS-06": "https://drive.google.com/file/d/1NlxwUqZ-9fjQpx28jiaMK7WQ7yr-VI-E/view",
    "SHS-08": "https://drive.google.com/drive/folders/19LKsbFWkJo7ZXTNr8vkAW2j1iF8Cgwhm",
    "SHS-11": "https://drive.google.com/file/d/1c5iWjch8zU9EB5Dn82qxWbb3qrvrwA-i/view",
    "SHS-13": "https://drive.google.com/file/d/1CCG-YxPhiJslLzeOuOM3iDU_0qq7tGIC/view",
    "SHS-16": "https://drive.google.com/file/d/1TupIg9WJFQpJxmnNJZge0CQbK_37WL6j/view",
    "SHS-20": "https://drive.google.com/file/d/1mlw7izsLjyZG0oPgpYaqyu8vRMzqG9jD/view",
    "SHS-21": "https://drive.google.com/file/d/13x80o_7kQavqdXlbvNl7E9o42W9tfecR/view",
    "SHS-22": "https://drive.google.com/file/d/1UpkGGvq5VWBIAc_Cqcw7TwscaqCN8Ucw/view",
    "SHS-23": "https://drive.google.com/file/d/1AApNNUOy1SGox_hrPmjNfs-AlAwCo2V6/view",
    "SHS-24": "https://drive.google.com/file/d/1J_IaZdMivrWRnLQMT_8uP4n3Azf1fcn7/view",
    "SHS-25": "https://drive.google.com/file/d/1twBXGso54l8OCjTH-MhMQnZUJBXXlyxb/view",
    "SHS-26": "https://drive.google.com/file/d/1DWueCFjvmbE_Pi-w8LwDcRDtzjnl5Eqn/view",
    "SPE-14": "https://drive.google.com/drive/folders/1bs7pAyXsH3TAEND20zSr43r_AgC_UzZX",
    "SPE-20": "https://drive.google.com/file/d/1vKuA5bI_P3jtLGrSu-xEx2uuAT0dNPCr/view",
    "SSS-02": "https://drive.google.com/file/d/1Hxx0wsHjTmbJ-LHcQvEA-g-MQNhJAIUG/view",
    "SSS-07": "https://drive.google.com/file/d/19I7SE_0trF5XCXCOicaegl4q03NpvdIt/view",
    "SSS-09": "https://drive.google.com/file/d/1WH0lXAtmpaoLH2SnXzlMcNq8mUxOc-Il/view",
    "SSS-18": "https://drive.google.com/file/d/1RupGYEmtKqazDhe0lbRT702HLPSniNY2/view",
    "SSS-19": "https://drive.google.com/file/d/1hW9M1PvRBNVTtikLYGGrm5Q_KfeV0eMN/view",
    "SUP-19": "https://drive.google.com/file/d/1s0BdUIg1ndCDeASDYdQKuoAW_0OjRUJ2/view",
    "SUP-20": "https://drive.google.com/drive/folders/1RvczVfwpvGYjOKzLH3YaW9z0EBM9_cDZ",
    "TRN-01": "https://drive.google.com/drive/folders/18Q4nLApPbCxvsVpFcQ41nlv-NBuU_V4Y",
    "TRN-02": "https://drive.google.com/drive/folders/1d5VLXilFWXtPOJep7O28smmgzv4t1uDb",
    "TRN-03": "https://drive.google.com/drive/folders/1h6k502jg5RNkL4KuvDhQhOL44XCORjM1",

}